In [1]:
import torch #no performance , max 45
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import numpy as np
from torch.distributions import Categorical


In [2]:
class enva(): 
    def __init__(self):
        self.height = 10
        self.width = 10
        self.num_obstacle_range =4
        self.num_obstacle_range_min = 3
        self.turn = 0
        self.position = int(round(self.width/2))
        self.map = np.array([[0.0 for j in range(self.width)] for i in range(self.height)])
        self.map[self.height-1][self.position] = 2. # 2.
        self.done = False
        
    def step(self, key):
        #obstacle down
        i = self.height - 2
        while(i>=0):
            self.map[i+1] = self.map[i]
            i -= 1
        #obstacle init(first_line)
        self.map[0] = np.array([0 for j in range(self.width)])
        #obstacle making
        num_obstacle = random.randrange(self.num_obstacle_range_min,self.num_obstacle_range)
        i = 0
        while(i < num_obstacle):
            position_obstacle = random.randrange(0,self.width)
            if (self.map[0][position_obstacle] == 1.0):
                continue
            self.map[0][position_obstacle] = 1.0
            i += 1
         #big_obstacle
        big_obstacle = random.randrange(0,20)
        if(big_obstacle == -3):###############################
            big_obstacle = random.randrange(1,self.width-1)
            for j in range(3):
                for k in range(-1,2):
                    self.map[j][big_obstacle + k] = 1.
        #player position
        reward = 1
        # 0 : 왼쪽, 1 : 가만히, 2 : 오른쪽
        if key == 0:
            if(self.position>0):
                self.position -= 1
            
        if key == 1:
            pass      
        if key == 2:
            if(self.position<self.width-1):
                self.position += 1
                  
        if (self.map[self.height-1][self.position] == 1.0):
                    reward = 0
                    self.done = True
                    #print("====Game Over====")
                        
        self.map[self.height-1][self.position] =  2. # 2.
        #reward(turn(time))
        self.turn += 1
        
        return torch.flatten(torch.tensor(self.map),0).numpy(), reward, self.done, _ 

    def reset(self):
        self.turn = 0
        self.position = round(self.width/2)
        self.map = np.array([[0.0 for j in range(self.width)] for i in range(self.height)])
        self.map[self.height-1][self.position] = 2.0
        self.done = False
        
        return torch.flatten(torch.tensor(self.map),0).numpy()

In [3]:
#Hyperparameters
learning_rate = 0.0005
gamma         = 0.98
lmbda         = 0.95
eps_clip      = 0.1
K_epoch       = 5
T_horizon     = 20


In [5]:
# PPO RNN 1 by 100

class PPO(nn.Module):
    def __init__(self):
        super(PPO, self).__init__()
        self.data = []

        self.fc0   = nn.Linear(100,128)
        self.rc1   = nn.RNN(input_size = 128, hidden_size = 128, num_layers = 2, batch_first=True) # 
        self.fc1   = nn.Linear(128,128)
        self.fc2   = nn.Linear(128,128)

        self.fc4   = nn.Linear(128,128)
        self.fc_pi = nn.Linear(128,3)
        self.fc_v  = nn.Linear(128,1)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def pi(self,x ,h0):
        x = x.view(-1,1,100)  
        size = x.size()[0] #
        #h0 = torch.zeros([2,size,128], dtype = torch.float)

        x = F.relu(self.fc0(x))
        x, hidden_state = self.rc1(x,h0) ##
        #hidden_state = torch.zeros([2,size,128], dtype = torch.float) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc4(x))
        x = self.fc_pi(x)
        prob = F.softmax(x, dim = 2)

        #torch.nan_to_num(prob, nan = 0.0)
        return prob, hidden_state
    
    def v(self, x, h0):
        x = x.view(-1,1,100)
        size = x.size()[0]
        #h0 = torch.zeros([2,size,128], dtype = torch.float)

        x = F.relu(self.fc0(x))
        x,_ = self.rc1(x,h0) ##
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc4(x))
        v = self.fc_v(x)
        
        return v
      
    def put_data(self, transition):
        self.data.append(transition)
        
    def make_batch(self):
        s_lst, a_lst, r_lst, s_prime_lst, prob_a_lst, h_in_lst, h_out_lst, done_lst = [], [], [], [], [], [], [], []

        for transition in self.data:
            s, a, r, s_prime, prob_a, h_in, h_out, done = transition
            
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            prob_a_lst.append([prob_a])
            h_in_lst.append(h_in.detach().numpy())
            h_out_lst.append(h_out.detach().numpy())
            done_mask = 0 if done else 1
            done_lst.append([done_mask])
        
        s,a,r,s_prime,done_mask, prob_a = torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
                                          torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
                                          torch.tensor(done_lst, dtype=torch.float), torch.tensor(prob_a_lst)
        self.data = []
        
        
        h_in_lst = torch.tensor(h_in_lst,dtype = torch.float)
        h_out_lst = torch.tensor(h_out_lst, dtype = torch.float)
        h_in_lst = h_in_lst.squeeze(2)
        h_out_lst = h_out_lst.squeeze(2)
        h_in_lst = h_in_lst.permute(1,0,2)
        h_out_lst = h_out_lst.permute(1,0,2)
        
        # num_layer X batch X hidden으로 만들기
        return s, a, r, s_prime, done_mask, prob_a, h_in_lst, h_out_lst
        
    def train_net(self):
        s, a, r, s_prime, done_mask, prob_a, h1, h2 = self.make_batch()

        h1 = h1.detach()
        h2 = h2.detach()
        
        
        td_target = r + gamma * self.v(s_prime, h2).squeeze(1) * done_mask
        delta = td_target - self.v(s, h1).squeeze(1)
        delta = delta.detach().numpy()

        advantage_lst = []
        advantage = 0.0
        for delta_t in delta[::-1]:
            advantage = gamma * lmbda * advantage + delta_t[0]
            advantage_lst.append([advantage])
        advantage_lst.reverse()
        advantage = torch.tensor(advantage_lst, dtype=torch.float) 
       
        for i in range(K_epoch):
        
            pi, _ = self.pi(s, h1) 
            pi_a = pi.squeeze(1).gather(1,a) 

            ratio = torch.exp(torch.log(pi_a) - torch.log(prob_a))  # a/b == exp(log(a)-log(b)) ##############

            
            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1-eps_clip, 1+eps_clip) * advantage
            
            loss = -torch.min(surr1, surr2) + F.smooth_l1_loss(self.v(s, h1).squeeze(1) , td_target.detach()) 
            
            self.optimizer.zero_grad()
            loss.mean().backward() # 여러 번 loss의 gradient를 구할 때
            self.optimizer.step()
        
def main():
    hidden = 128
    env = enva()
    model = PPO()
    #model.load_state_dict(torch.load('ppo_ddong_with_dongmin.pt'))

    score = 0.0
    print_interval = 500
    best_score = 0.0
    one_score = 0.0
    
    for n_epi in range(60000):
        s = env.reset()
        done = False
        h_out = torch.zeros([2,1,hidden],dtype = torch.float) #num_layers X batch X hidden size
        while not done:
            for t in range(T_horizon):
                h_in = h_out
                prob, h_out = model.pi(torch.from_numpy(s).float(), h_in)
                prob = prob.view(-1) 
                m = Categorical(prob)
                a = m.sample().item()
                s_prime, r, done, info = env.step(a)
                
                model.put_data((s, a, r, s_prime, prob[a].item(), h_in, h_out, done))
                s = s_prime
                
                score += r
                one_score += r
                if done:
                    break
            
            model.train_net()
            
        if best_score <= one_score:
            best_score = one_score
        #    print("... save model ...")
            print(f"best score : {one_score}")
        #    torch.save(model.state_dict(),'ppo_ddong_with_dongmin.pt')
            
        one_score = 0
            
        if n_epi%print_interval==0 and n_epi!=0:
            print("# of episode :{}, avg score : {:.1f}".format(n_epi, score/print_interval))
            score = 0.0
    
    return best_score

if __name__ == '__main__':
    best_score = main()
    print(f"final best_score : {best_score}")
    


best score : 13.0
best score : 15
best score : 21
best score : 23
best score : 23
best score : 26
# of episode :500, avg score : 11.4
best score : 30
# of episode :1000, avg score : 11.3
best score : 32
# of episode :1500, avg score : 11.4
best score : 34
# of episode :2000, avg score : 11.5
best score : 36
# of episode :2500, avg score : 11.6
best score : 38
# of episode :3000, avg score : 11.4
# of episode :3500, avg score : 11.3
# of episode :4000, avg score : 11.2
# of episode :4500, avg score : 11.3
# of episode :5000, avg score : 11.6
# of episode :5500, avg score : 11.4
# of episode :6000, avg score : 11.1
# of episode :6500, avg score : 11.2
# of episode :7000, avg score : 11.5
# of episode :7500, avg score : 11.3
# of episode :8000, avg score : 11.6
# of episode :8500, avg score : 11.2
# of episode :9000, avg score : 11.4
# of episode :9500, avg score : 11.2
# of episode :10000, avg score : 11.2
# of episode :10500, avg score : 11.6
# of episode :11000, avg score : 11.3
# of e

KeyboardInterrupt: 